In [ ]:
import babelnet as bn
from babelnet import BabelSynsetID, Language
from babelnet.data.relation import BabelPointer

In [ ]:
from  zerorpc import TimeoutExpired, LostRemote

In [ ]:
bn.version()

In [ ]:
### Root searcher
visited = set()

max_visits, n = 20000, 0
start_synset_id = 'bn:00015267n'
start_synset = bn.get_synset(BabelSynsetID(start_synset_id))
q = [start_synset]

with open(f'candidate_roots_{max_visits}_{start_synset_id}.txt', 'w') as nohypfile:
    while q and n <= max_visits:
        synset = q.pop(0)
        n += 1
        hypernym_edges = synset.outgoing_edges(BabelPointer.ANY_HYPERNYM)
        if hypernym_edges == []:
            print(f'Possible root node: {synset.id}')
            nohypfile.write(f'{str(synset.id)},{synset.main_sense().full_lemma}\n')
        for edge in hypernym_edges:
            if edge.id_target not in visited:
                q.append(bn.get_synset(edge.id_target))
                visited.add(edge.id_target)

In [ ]:
for id_lemma in open('../log/candidate_roots.txt', 'r').readlines()[2:]:
    id, lemma = id_lemma.strip().split(',')
    synset = bn.get_synset(BabelSynsetID(id))
    print(f'{id},{lemma},{len(synset.outgoing_edges())},{len(synset.outgoing_edges(BabelPointer.ANY_HYPERNYM))}')